# 작업 1유형 

20L가격과 5L가격이 모두 0원이 아닌 데이터만 필터를 한 후, 각 row별로 20L가격과 5L가격의 차이를 ‘차이가격’ 이라 부른다고 하자. 시도명 별 차이가격의 평균가격을 비교할때 그 값이 가장 큰 금액을 반올림하여 소숫점 이하 1자리까지 구하여라

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p1_1_.csv')
df.head(2)

temp = df[(df['20L가격']!=0) & (df['5L가격']!=0)]
temp['가격차이'] = df['20L가격'] - df['5L가격']
result = temp.groupby(['시도명']).가격차이.mean().sort_values(ascending=False)
print(int(result[0]))

619


C:\Users\choic\AppData\Local\Temp\ipykernel_27600\1183479388.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['가격차이'] = df['20L가격'] - df['5L가격']


Q2. BMI는 몸무게(kg) / (키(M) * 키(M)) 로 정의 된다. 
- 초고도 비만은 BMI 25이상
- 고도 비반은 BMI 25미만 - 23이상
- 정상은 23미만 - 18.5이상 
- 저체중은 18.5미만으로 정의 된다. 
- 주어진 데이터에서 초고도비만 인원 + 저체중 인원 의 숫자는?

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p1_2_.csv')
df.head(5)

df['bmi'] = (df['weight(kg)']/(df['height(cm)']*df['height(cm)']))*10000
cond1 = df[(df['bmi']>=25)]
cond2 = df[df['bmi']<18.5]

print(cond1.shape[0]+cond2.shape[0])

8998


Q3. 순유입인원은 초중고 도내,도외 전입인원에서 초중고 도내, 도외 전출인원을 뺀값이다. 각년도별로 가장 큰 순유입인원을 가진 지역구의 순유입인원을 구하고 전체 기간의 해당 순유입인원들의 합을 구하여라

In [3]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p1_3.csv')
df.head(5)

,지역,초등학교_전출_도내,초등학교_전출_도외,초등학교_전입_도내,초등학교_전입_도외,중학교_전출_도내,중학교_전출_도외,중학교_전입_도내,중학교_전입_도외,고등학교_전출_도내,고등학교_전출_도외,고등학교_전입_도내,고등학교_전입_도외,년도
0,종로구,221,132,255,126,63,71,104,73,203,72,166,75,2012
1,중구,295,139,203,100,66,43,58,44,156,37,104,39,2012
2,용산구,380,269,302,174,134,78,91,106,102,66,113,48,2012
3,성동구,643,344,376,221,154,103,98,82,179,37,153,41,2012
4,광진구,560,382,595,312,165,158,158,118,115,70,154,66,2012


In [4]:
df['전출'] = df['초등학교_전출_도내']+df['중학교_전출_도내']+df['고등학교_전출_도내']\
        +df['초등학교_전출_도외']+df['중학교_전출_도외']+df['고등학교_전출_도외']

df['전입'] = df['초등학교_전입_도내']+df['중학교_전입_도내']+df['고등학교_전입_도내']\
        +df['초등학교_전입_도외']+df['중학교_전입_도외']+df['고등학교_전입_도외']

df['순유입인원'] = df['전입'] - df['전출']
result = df.groupby('년도')['지역','순유입인원'].max().sum()
result
print(13853)

13853


C:\Users\choic\AppData\Local\Temp\ipykernel_27600\2071412217.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = df.groupby('년도')['지역','순유입인원'].max().sum()


# 작업2유형

예측 변수 price, test.csv에 대해 ID별로 price 값을 예측하여 제출, 제출 데이터 컬럼은 ID와 price 두개만 존재해야함. 평가지표는 rmse

In [16]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p2_test_.csv')

y = train['price']
test_id = test['ID']
#=====전처리 한 다음=======
train = train.drop(columns=['ID','price'])
test = test.drop(columns='ID')

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

train['model'] = encoder.fit_transform(train['model'])
test['model'] = encoder.fit_transform(test['model'])

train['transmission'] = encoder.fit_transform(train['transmission'])
test['transmission'] = encoder.fit_transform(test['transmission'])

train['fuelType'] = encoder.fit_transform(train['fuelType'])
test['fuelType'] = encoder.fit_transform(test['fuelType'])

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

x_train,x_test,y_train,y_test = train_test_split(train,y,test_size=0.2)

rf = RandomForestRegressor()
rf.fit(x_train,y_train)
pred_rf = rf.predict(x_test)
print('test rmse', np.sqrt(mean_squared_error(y_test,pred_rf)))

predict = rf.predict(test)
predict

submission = pd.DataFrame({'ID':test_id,
                          'price':predict})
submission.to_csv('./12345.csv',index=False)

submission

test rmse 2594.412013792516


,ID,price
0,0,4571.5000
1,2,66658.2700
2,14,17301.2700
3,15,13299.7500
4,18,15282.7500
...,...,...
3291,13096,27199.2100
3292,13098,24929.9100
3293,13101,19608.0500
3294,13109,16739.0000


# 작업3유형 

Q1.3 그룹의 데이터에 대해 크루스칼-왈리스 검정을 사용하여 검정 통계량을 반올림하여 소숫점 이하 3자리까지 구하여라

In [31]:
import pandas as pd 
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e5_p3_2.csv')
df.head(3)

,ID,value
0,A,84.179434
1,A,93.971003
2,A,62.140957


In [36]:
a = df[(df['ID']=='A')]
b = df[(df['ID']=='B')]
c = df[(df['ID']=='C')]

In [39]:
from scipy.stats import kruskal
s,p = kruskal(a['value'],b['value'],c['value'])
print(round(s,3))

6.521


Q2.3 그룹의 데이터에 대해 크루스칼-왈리스 검정을 사용하여 p-value를 반올림하여 소숫점 이하 3자리까지 구하여라. 귀무가설과 대립가설중 0.05 유의수준에서 유의한 가설을 출력하라

In [42]:
print(round(p,3))
print('대립')

0.038
대립


In [17]:
help('scipy.stats')

Help on package scipy.stats in scipy:

NAME
    scipy.stats - .. _statsrefmanual:

DESCRIPTION
    Statistical functions (:mod:`scipy.stats`)
    
    .. currentmodule:: scipy.stats
    
    This module contains a large number of probability distributions,
    summary and frequency statistics, correlation functions and statistical
    tests, masked statistics, kernel density estimation, quasi-Monte Carlo
    functionality, and more.
    
    Statistics is a very large area, and there are topics that are out of scope
    for SciPy and are covered by other packages. Some of the most important ones
    are:
    
    - `statsmodels <https://www.statsmodels.org/stable/index.html>`__:
      regression, linear models, time series analysis, extensions to topics
      also covered by ``scipy.stats``.
    - `Pandas <https://pandas.pydata.org/>`__: tabular data, time series
      functionality, interfaces to other statistical languages.
    - `PyMC3 <https://docs.pymc.io/>`__: Bayesian statistica